In [15]:
# Import required packages
import sys
# !{sys.executable} -m pip install seaborn
from glob import glob
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from shutil import copy,move
from distutils.dir_util import copy_tree


In [21]:
files = sorted(glob('./5/*/*results.json'))
baseline = "../BSE-calc-test/5qubit/A2_HWE-CNOT/A2_HWE-CNOT_results.json"

In [22]:
glob('5/*/*_results.json')

['5/0.3/A2_HWE-CNOT_results.json',
 '5/0.5/A2_HWE-CNOT_results.json',
 '5/0.7/A2_HWE-CNOT_results.json',
 '5/0.1/A2_HWE-CNOT_results.json',
 '5/0.8/A2_HWE-CNOT_results.json']

In [23]:

quantum_results = []
quantum_data = []
for i in files:
    print(i)
    with open(i,'r') as f:
        data = json.load(f)
    ratio = os.path.basename(os.path.dirname(os.path.dirname(i)))
    quantum_data.append((ratio,data['R2_train'],data['R2_test']))
    quantum_results.append(('A2_HWE-CNOT',ratio,'R2_train',data['R2_train']))
    quantum_results.append(("A2_HWE-CNOT",ratio,'R2_test',data['R2_test']))

quantum_data = np.array(quantum_data).astype(float)

./5/0.1/A2_HWE-CNOT_results.json
./5/0.3/A2_HWE-CNOT_results.json
./5/0.5/A2_HWE-CNOT_results.json
./5/0.7/A2_HWE-CNOT_results.json
./5/0.8/A2_HWE-CNOT_results.json


In [24]:
quantum_data

array([[5.        , 0.09047927, 0.09584694],
       [5.        , 0.09160022, 0.09734892],
       [5.        , 0.08251446, 0.09164304],
       [5.        , 0.0886865 , 0.09508284],
       [5.        , 0.08333327, 0.08419636]])

In [9]:

with open(baseline,'r') as f:
    quantum_base = json.load(f)

In [12]:
'../classical_testing/BSE/16_0.1/scores.json'

'../classical_testing/BSE/16_0.1/scores.json'

In [10]:

trainsize=[ 0.1,0.3, 0.5, 0.7,0.8]
classical_models=['ridge', 'lasso', 'elastic', 'knn', 'rfr', 'grad', 'svr', 'krr', 'gpr'
]

classical_results=[]

for models in classical_models:
    for ts in trainsize:
        with open(f"../classical_testing/BSE/{ts}_{models}_scores.json",'r') as f:
            classical_dict=json.load(f)
        classical_results.append((models,ts,'R2_train',classical_dict['R2_train']))
        classical_results.append((models,ts,'R2_test',classical_dict['R2_test']))
            

FileNotFoundError: [Errno 2] No such file or directory: '/Users/grierjones/qregress/classical_testing/linear/0.1_ridge_scores.json'

In [11]:
classical_df=pd.DataFrame(classical_results,columns=['model','ratio','set','value'])

In [ ]:

quantum_df=pd.DataFrame(quantum_results,columns=['model','ratio','set','value'])

In [ ]:
combined_df = pd.concat([classical_df,quantum_df])

In [ ]:
quantum_df['ratio']

In [ ]:
classical_df[classical_df['ratio']>0.1]['value'].describe()

In [ ]:
quantum_df[quantum_df['ratio']>"0.1"]['value'].describe()

In [ ]:
for ts in trainsize:
    print(ts)
    for m in classical_models:
        cdf=classical_df[(classical_df['ratio']==ts)&(classical_df['model']==m)].set_index('set')
        qdf=quantum_df[quantum_df['ratio']==f'{ts}'].set_index('set')
        print(m,all(cdf['value']>=qdf['value']))
        if cdf['value'].loc['R2_test']>=qdf['value'].loc['R2_test']:
            print(cdf['value'].loc['R2_test'],qdf['value'].loc['R2_test'])
    print()

In [ ]:
palette=sns.color_palette('Paired',10)
model_pal = dict(zip(set(combined_df['model']),palette))


SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
plt.figure(figsize=(8,5))
ax = sns.lineplot(data=combined_df,x='ratio',y='value',style='set',hue='model',palette=model_pal, markers=True)
ax.legend_.remove()

ins = ax.inset_axes([0.7,0.7,0.2,0.2])
sns.lineplot(data=combined_df[(combined_df['ratio']==0.1)&(combined_df['value']>0.8)],x='ratio',y='value',style='set',hue='model',palette=model_pal, markers=True,ax=ins)
ins.set_ylim(0.8,1.05)
ins.set_ylabel('R$^{2}$')
ins.set_xlabel('Training Set Ratio')
ins.set_xticks([0.1])
ins.get_legend().remove()



# Add separate legend for hue (category)
handles, labels = ax.get_legend_handles_labels()
hue_legend = plt.legend(handles[1:11]+handles[12:], labels[1:11]+labels[12:], title='Model/Dataset', bbox_to_anchor=(0.3, 0.41), borderaxespad=0,ncols=3)

# Add separate legend for style
# style_legend = plt.legend(handles[12:], labels[12:], bbox_to_anchor=(1, 0.2), loc='upper left', borderaxespad=0.5)

# Add the hue legend back so both are displayed
plt.gca().add_artist(hue_legend)


ax.set_ylim(0,1.05)
ax.set_xlim(0.05,0.85)
ax.set_ylabel('R$^{2}$')
ax.set_xlabel('Training Set Ratio')
plt.tight_layout()
plt.savefig('/Users/grierjones/qregress/images/linear_learning_curves.png', bbox_inches='tight', pad_inches=0.1)
plt.show()